# Using Dash in Jupyterlab

To make Dash work well with a fairly normal analysis workflow, in addition to `dash` we need two additional packages:
- `jupyter-dash` -- Allows re-running app building/serving without restarting the jupyter kernel
- `dash-extensions` -- Provides a way to keep outputs serverside (ServersideOutput) which makes datatypes more flexible
- `jupyter-server-proxy` -- Allows viewing the Dash app while connected remotely

Because both `jupyter-dash` and `dash-extensions` requires using their own subclass of the `Dash` class, we will combine them and use combined class instead.

These packages have very loose dependencies (i.e. they don't specify what they need) but apparently very strict requirements (i.e. they fail if they don't have exactly what they need)

Below is a list of the main packages that are relevant and versions that definitely work together:

In [1]:
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc
from dash import Input, Output, State
from dash_extensions.enrich import DashProxy, ServersideOutput, ServersideOutputTransform, ServerStore

class MyDash(JupyterDash, DashProxy):    
    """Allow use of dash-extensions while maintaining the jupyter-dash behavior
    
    When asking for a method on this class it will look in JupyterDash first, then DashProxy if it didn't find it in JupyterDash.
    """
    pass

In [2]:
raise Exception(f'\n\n>>>>> Note: `JupyterDash.infer_jupyter_proxy_config()` has to be run with Shift+Enter (not the Fast Forward button) <<<<<\n>>>>> This exception is here to enforce that <<<<<')

Exception: 

>>>>> Note: `JupyterDash.infer_jupyter_proxy_config()` has to be run with Shift+Enter (not the Fast Forward button) <<<<<
>>>>> This exception is here to enforce that <<<<<

In [3]:
# This allows the dash app to work when connected remotely
# Note: For some reason, running this line is incomptible with using the `lab_black` extension. (and lab_black cannot be loaded later)
%unload_ext lab_black
JupyterDash.infer_jupyter_proxy_config()

The lab_black extension is not loaded.


In [4]:
# Make a couple of functions to help later
def make_app() -> MyDash:
    """Make a new instance of the dash app (then add layout and callbacks)"""
    app = MyDash(__name__, transforms=[ServersideOutputTransform()], external_stylesheets=[dbc.themes.BOOTSTRAP])
    return app

def add_label(component, label: str):
    """Combine a label with a component when placing in layout"""
    return html.Div([label, component])

## Extremely basic tests to check the dash app works at all

In [6]:
# Super basic test that the server runs with the most basic layout
app = make_app()
app.layout = html.Div('test layout')

# Uncomment to check this test works
# app.run_server(mode='inline')

In [8]:
# Test that the simplest callback works
app = make_app()

output = html.Div(id='df-out')
in_a = dbc.Input('in-a', value=0)
app.layout = html.Div([in_a, output])

@app.callback(Output(output.id, 'children'), Input(in_a.id, 'value'))
def test_output(a):
    return a

# Uncomment to check this test works
# app.run_server(mode='inline')

## Example of working with more complex functions

In [9]:
import pandas as pd

### Make/test a function as if it was being used in a normal analysis page

In [10]:
# Now test a basic function that doesn't return a json serializable output (e.g. that would usually be required by dash)
def make_df_test(a, b):
    """E.g. Some part of normal data processing"""
    # Make sure inputs are floats (dash passes strings for some types of input)
    a, b = float(a), float(b)
    df = pd.DataFrame(data=[a, b])
    return df
    
# Use the function for normal processing or for testing it works as expected
test_df = make_df_test(3,4)
test_df

,0
0,3.0
1,4.0


### Use that function in a dash app

In [15]:
app = make_app()

# Make component to store output of make_df_test
df_store = dcc.Store('df-store')

# Make component to display output of make_df_test
output = dcc.Markdown(id='df-out')

# Make components for the inputs of the make_df_test
in_a = dbc.Input(id='in-a', value=0)
in_b = dcc.Dropdown(id='in-b', options=[1,2,3,4,5], value=3)

# Put those components into a layout (I'll add helper functions to make this easier)
app.layout = html.Div([add_label(in_a, 'Input A'), add_label(in_b, 'Input B'), df_store, html.H2('Output 1'), output])

# Define a function that displays the output of make_df_test
def show_df(df: pd.DataFrame):
    """Convert df to something to show in Dash App"""
    return df.to_markdown()
    
# Attach a callback to call make_df_test (Note: ServersideStore keeps the output on the server and allows for any pickleable object (not as strict as Json Serializeable)
app.callback(ServersideOutput(df_store.id, 'data'), Input(in_a.id, 'value'), Input(in_b.id, 'value'))(make_df_test)

# Attach the callback to show the output of make_df_test
app.callback(Output(output.id, 'children'), Input(df_store.id, 'data'))(show_df)

# Uncomment to check this test works at this stage
# app.run_server(mode='inline')

<function __main__.show_df(df: pandas.core.frame.DataFrame)>

### Example of how this would extend to later functions that rely on earlier ones

In [16]:
# Example of using another function that takes the output of the first
def do_something_with_df(df: pd.DataFrame, multiplier):
    return df.apply(lambda x: x*multiplier)

test_df = make_df_test(4,5)
updated_test_df = do_something_with_df(test_df, 5)
updated_test_df

,0
0,20.0
1,25.0


In [17]:
# Add this next function to dash app

# Make new inputs
multiplier_input = dcc.Slider(id='slider-multiplier', min=1, max=10, step=1, value=1)

# Make new outputs
multiplied_store = dcc.Store(id='store-multiplied')
multiplied_output = dcc.Markdown(id='md-multiplied_out')

# Now add to the dash app (adding to the existing layout)
app.layout = html.Div([add_label(in_a, 'in_a'), add_label(in_b, 'in_b'), df_store, html.H2('Output 1'), output, html.Hr(), add_label(multiplier_input, 'multiplier'), multiplied_store, html.H2('Output 2'), multiplied_output])

# Attach the callback to run the process
app.callback(ServersideOutput(multiplied_store.id, 'data'), Input(df_store.id, 'data'), Input(multiplier_input.id, 'value'))(do_something_with_df)

# Attach a callback to view the new output (in this case, can use the same function as last time)
app.callback(Output(multiplied_output.id, 'children'), Input(multiplied_store.id, 'data'))(show_df)

# Run server to test (Comment out if testing further down)
app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/

